In [35]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct


import pandas as pd
import openai

In [3]:
client = QdrantClient(
    url="http://localhost:6333",
)

# size is unchangeable
client.create_collection(
    collection_name="Amazon-items-collection-00",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

True

In [10]:
import pandas as pd

df_items = pd.read_json("../data/meta_Electronics_1000.jsonl", lines=True)

df_items.head(3)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,available_date
0,All Electronics,"Adjustable Metal Stand for E cho Show 15, Tilt...",4.5,111,[The VMEI adjustable metal bracket is made of ...,[],36.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Small footprint, minimalist design...",VMEI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'VMEI', 'Color': 'Black', 'Room Type...",B0B13T2GY5,NaN,2022-05-11
1,Computers,"HP Laptop, 15.6"" HD Touchscreen, AMD Athlon Go...",4.4,205,[【High Speed RAM And Enormous Space】8GB high-b...,[PConline365 sells computers with professional...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],HP,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'HP', 'Color': 'Silver', 'Room Type'...",B08DC5KLJZ,NaN,2022-03-24
2,Amazon Home,50 Pcs Groot Stickers for Water Bottle Waterpr...,4.7,156,[🏆Groot Stickers:Anime theme party favors for ...,[Groot Stickers Quantity: 50 Pcs/pack.Non-dupl...,5.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': '200-Piece Stickers Pack for Laptop...,Fcelery,"[Electronics, Computers & Accessories, Laptop ...","{'Brand': 'Fcelery', 'Color': 'Groot', 'Room T...",B0B4QQSRY3,NaN,2022-06-22


In [11]:
def preprocess_data(row):
    return f"{row['title']} {' '.join(row['features'])}"

In [13]:
df_items["preprocessed_data"] = df_items.apply(preprocess_data, axis=1)

df_items.head(2)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,available_date,preprocessed_data
0,All Electronics,"Adjustable Metal Stand for E cho Show 15, Tilt...",4.5,111,[The VMEI adjustable metal bracket is made of ...,[],36.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Small footprint, minimalist design...",VMEI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'VMEI', 'Color': 'Black', 'Room Type...",B0B13T2GY5,NaN,2022-05-11,"Adjustable Metal Stand for E cho Show 15, Tilt..."
1,Computers,"HP Laptop, 15.6"" HD Touchscreen, AMD Athlon Go...",4.4,205,[【High Speed RAM And Enormous Space】8GB high-b...,[PConline365 sells computers with professional...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],HP,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'HP', 'Color': 'Silver', 'Room Type'...",B08DC5KLJZ,NaN,2022-03-24,"HP Laptop, 15.6"" HD Touchscreen, AMD Athlon Go..."


In [15]:
df_sample = df_items.sample(n=50, random_state=42)

len(df_sample)

50

In [18]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=[text],
        model=model,
    )
    return response.data[0].embedding

In [21]:
len(get_embedding("hi"))

1536

In [22]:
import numpy as np

def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [53]:
vec_hi     = get_embedding("Cześć! Jak się czujesz?")
vec_hello  = get_embedding("Hej, jak się dziś masz?")
vec_banana = get_embedding("Zupełnie niezwiąany temat")
vec_ola    = get_embedding("Przestępca zabił człowieka")

print("hi vs hello:", cosine_similarity(vec_hi, vec_hello))   # np. 0.92
print("hi vs Ola:", cosine_similarity(vec_hi, vec_ola))   # np. 0.92
print("hi vs banana:", cosine_similarity(vec_hi, vec_banana)) # np. 0.55

hi vs hello: 0.7531083731949886
hi vs Ola: 0.20940790991639888
hi vs banana: 0.33334121143613715


In [39]:
data_to_embed = df_sample["preprocessed_data"].tolist()

pointstructs = []
for i, data in enumerate(data_to_embed):
    embedding = get_embedding(data)
    pointstruct = PointStruct(
        id=i,
        vector=embedding,
        payload={"text": data}
    )
    pointstructs.append(pointstruct)

pointstructs

[PointStruct(id=0, vector=[-0.004300535190850496, -0.03781024366617203, -0.057982463389635086, 0.019584286957979202, 0.0200303066521883, -0.00637097517028451, -0.04176359251141548, 0.04472353681921959, 0.0312618725001812, -0.020415503531694412, 0.02781536430120468, 0.025564996525645256, -0.012670992873609066, -0.02090206928551197, 0.023456541821360588, -0.037952158600091934, -0.0039026658050715923, -0.00031170641887001693, -0.027957279235124588, 0.0067815156653523445, -0.03184980899095535, 0.05696878582239151, -0.012143880128860474, -0.026801684871315956, -0.026011014357209206, 0.01987825334072113, -0.020182358101010323, 0.032214730978012085, 0.024794599041342735, -0.0474807433784008, 0.010745001956820488, -0.03926994279026985, -0.013471799902617931, -0.03308649733662605, -0.03191062808036804, 0.016198597848415375, -0.04068909212946892, -0.022300947457551956, -0.0021477332338690758, -0.00677644694224, 0.008079024963080883, 0.026538128033280373, 0.04188523441553116, -0.01857060752809047

In [40]:
client.upsert(
    collection_name="Amazon-items-collection-00",
    wait=True,
    points=pointstructs
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [44]:
def retrieve_data(query):
    query_embedding = get_embedding(query)
    results = client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=10,
    )
    return results

In [50]:
retrieve_data("headphones").points

[ScoredPoint(id=1, version=0, score=0.49889624, payload={'text': 'Bang & Olufsen Beoplay HX – Comfortable Wireless ANC Over-Ear Headphones - Dark Maroon EXTREMELY COMFORTABLE HEADPHONES. Designed with extended wear in mind, Beoplay HX have a comfortable, lightweight build and elegant touch control. POWERFUL ANC. A new generation of hybrid active noise cancellation compensates for background noise, allowing you to fully immerse yourself in sound. LONG-LASTING BATTERY. With up to 30 hours of playtime on a single charge, Beoplay HX push the boundaries to give you more time to enjoy your music. TIMELESS ELEGANCE. Authentic Scandinavian aesthetics ensure a minimalist design that complements a busy lifestyle and strong personal style. INSPIRING AUDIO. Well-balanced bass and treble performance combined with a clear midrange means you can enjoy music the way the artist intended.'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=6, version=0, score=0.4942053, payload={'text': '